In [8]:
import os


In [12]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'WisdomAnalyst'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '781aa8b98086477f57088d72da8f137a4686f662'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/WisdomAnalyst/End-to-end-Machine-Learning-Project-with-Mlflow.mlflow'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column:str 
    mlflow_url:str


In [17]:
from Mlproject.constants import *
from Mlproject.utils.common import read_yaml, create_directories
import logging
import json
from pathlib import Path
from box import ConfigBox

logging.basicConfig(level=logging.INFO, format='[%(asctime)s]: %(message)s:')

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")


In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = ConfigBox(read_yaml(config_filepath))
        self.params = ConfigBox(read_yaml(params_filepath))
        self.schema = ConfigBox(read_yaml(schema_filepath))

        logging.info(f"Config contents: {self.config}")
        create_directories([self.config.artifacts_root])

    def get_model_config(self) -> ModelEvaluationConfig:
        config = self.config.model_Evaluation
        params = self.params.Elasticnet
        schema = self.schema.TARGET_COLUMNS

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_url='https://dagshub.com/WisdomAnalyst/End-to-end-Machine-Learning-Project-with-Mlflow.mlflow'
        )

        return model_evaluation_config